In [6]:
import keras
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from helper import load_data
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import applications


In [7]:
IMAGE_SIZE = 150
CHANNELS = 3
NUM_EPOCH = 30
LEARN_RATE = 1.0e-4

In [8]:
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS))

 7340032/58889256 [==>...........................] - ETA: 20:38

IOError: Unable to open file (Truncated file: eof = 7328280, sblock->base_addr = 0, stored_eoa = 58889256)

The question raised is why rescale is 1./255 and why we need this before training neural network. For colorful image, every image contains three maps: Red, Green and Blue, and all the pixel still in the range 0~255.
Rescale is important to treat all images in the same manner: some images are high pixel range, some are low pixel range. Scaling every images to the same range [0,1] will make images contributes more evenly to the total loss. 

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255.)
train_generator = train_datagen.flow_from_directory(
        'data/train',
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=32,
        class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
        'data/valid',
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=32,
        class_mode='categorical')

Found 2000 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


In [4]:
def cnn_model():
    
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(IMAGE_SIZE,IMAGE_SIZE,CHANNELS)))    
    model.add(Conv2D(32, (3, 3), activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(3, activation='softmax'))
    
    model.summary()
    
    return model

In [5]:
model = cnn_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 148, 148, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 74, 74, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
__________

In [6]:
checkpoint = ModelCheckpoint('best_model_simple.h5',  # model filename
                             monitor='val_loss', # quantity to monitor
                             verbose=0, # verbosity - 0 or 1
                             save_best_only= True, # The latest best model will not be overwritten
                             mode='auto') # The decision to overwrite model is made 
                                          # automatically depending on the quantity to monitor

In [7]:
model.compile(loss='categorical_crossentropy', # Better loss function for neural networks
              optimizer=Adam(lr=1.0e-4), # Adam optimizer with 1.0e-4 learning rate
              metrics = ['accuracy']) # Metrics to be evaluated by the model

In [ ]:
model.fit_generator(
        train_generator,
        epochs=NUM_EPOCH,
        callbacks=[checkpoint],
        validation_data=validation_generator)
model.save_weights('first_try.h5')

Epoch 1/30
29/63 [============>.................] - ETA: 10:03 - loss: 1.0232 - acc: 0.6185